In [5]:
import os
import numpy as np 
import pandas as pd 


from sklearn import model_selection, preprocessing 
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler

import xgboost as xgb
from xgboost import XGBClassifier

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from typing import Any, Dict, Union
from yellowbrick import model_selection as ms
from yellowbrick.model_selection import validation_curve

import mlflow
import urllib
import zipfile

In [13]:
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/'
feature_df = pd.read_csv(f'{results_path}acceptedfeatures.csv')
features = feature_df['AcceptedFeatures'].to_list()
X_train = pd.read_csv(f'{results_path}X_train.csv')
X_test = pd.read_csv(f'{results_path}X_test.csv')
y_train = pd.read_csv(f'{results_path}y_train.csv')
y_test = pd.read_csv(f'{results_path}y_test.csv')

In [14]:
ex_id = mlflow.create_experiment(name = 'ex3', artifact_location = 'ex2path')
mlflow.set_experiment(experiment_name = 'ex3')
with mlflow.start_run():
    params = {'random_state': 42}
    rounds = [{'max_depth': hp.quniform('max_depth', 1, 12, 1),
              'min_child_weight': hp.loguniform('min_child_weight', -2, 3)},
             {'subsample': hp.uniform('subsample', 0.5, 1),
             'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)},
             {'gamma': hp.loguniform('gamma', -10, 10)},
             {'learning_rate': hp.loguniform('learning_rate', -7, 0)}
             ]

### Add grouped k fold validation 

In [ ]:
for round in rounds:
    params = {**params, **round}
    trials = Trials()
    best = fmin(fn = lambda space: xhelp.hyperparameter_tuning(
    space, X_train, y_train, X_test, y_test),
               space = params,
               algo = tpe.suggest,
               max_evals = 10,
               trials = trials,
               timeout = 60*5)
    params = {**params, **best}
    for param, val in params.items():
        mlflow.log_param(param, val)
    params['max_depth'] = int(params['max_depth'])
    xg = xgb.XGBClassifier(eval_metric = 'logloss', early_stopping_rounds = 50, **params)
    #xg.fit(X_train, y_train,
    #      X_test, y_test)